In [1]:
import requests
import pandas as pd
import os

# Foursquare

#### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
bike_stations = pd.read_csv('bike_stations.csv')

url = "https://api.foursquare.com/v3/places/search"

headers = {'accept' : 'application/JSON',
           'Authorization' : os.getenv('FOURSQUARE_API_KEY')}

def request_places(row):
    params = {'ll' : f"{row['latitude']},{row['longitude']}",
             'radius' : 1000,
             'query' : 'resturant'}
    response = requests.get(url, headers=headers, params=params)
    res = response.json()
    places = pd.Series([row['station_name'], res])
    return places

foursquare_places = bike_stations.apply(request_places, axis=1)

#### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [102]:
nearest_station_name = []
resturant_name = []
distance = []
address = []
categories = []

def get_details(row):
    for resturant in row[1]['results']:
        resturant_name.append(resturant['name'])
        distance.append(resturant['distance'])
        address.append(resturant['location']['address'])
        nearest_station_name.append(row[0])

        resturant_categories = []
        for category in resturant['categories']:
            resturant_categories.append(category['name'])
        categories.append(', '.join(resturant_categories))

foursquare_places.apply(get_details, axis=1)

foursquare_resturants = {
                         'resturant_name' : resturant_name,
                         'categories' : categories,
                         'address' : address,
                         'station_name' : nearest_station_name,
                         'distance' : distance,
                        }

Put your parsed results into a DataFrame

In [104]:
foursquare_resturants = pd.DataFrame(foursquare_resturants)

foursquare_resturants.head()

,resturant_name,categories,address,station_name,distance
0,Pizzeria Alle Scalette,Pizzeria,Viale Curtatone 18,Curtatone,22
1,Grom,"Ice Cream Parlor, Restaurant",13 Via Banchi di Sopra 11,Curtatone,367
2,Osteria L'Osteria,Italian Restaurant,Via dei Rossi 79,Curtatone,342
3,Il Masgalano,"Café, Gelato Shop",Via Camporegio 1,Curtatone,156
4,Ristorante San Domenico,Restaurant,Via Camporegio 17,Curtatone,170


# Yelp

#### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [105]:
url = "https://api.yelp.com/v3/businesses/search"

headers = {'accept' : 'application/JSON',
           'Authorization' : 'Bearer ' + os.getenv('YELP_KEY')}

def request_places(row):
    params = {'latitude' : row['latitude'],
              'longitude' : row['longitude'],
              'radius' : 1000,
              'categories' : 'resturants'
             }
    response = requests.get(url, headers=headers, params=params)
    res = response.json()
    places = pd.Series([row['station_name'], res])
    return places

yelp_places = bike_stations.apply(request_places, axis=1)



#### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [106]:
nearest_station_name = []
resturant_name = []
distance = []
address = []
review_count = []
rating = []
categories = []

def get_details(row):
    for resturant in row[1]['businesses']:
        resturant_name.append(resturant['name'])
        distance.append(resturant['distance'])
        address.append(resturant['location']['address1'])
        review_count.append(resturant['review_count'])
        rating.append(resturant['rating'])

        resturant_categories = []
        for category in resturant['categories']:
            resturant_categories.append(category['title'])
        categories.append(', '.join(resturant_categories))

        nearest_station_name.append(row[0])
        

yelp_places.apply(get_details, axis=1)

yelp_resturants = {
                    'resturant_name' : resturant_name,
                    'categories' : categories,
                    'address' : address,
                    'review_count' : review_count,
                    'rating' : rating,
                    'station_name' : nearest_station_name,
                    'distance' : distance
                    }
                    

#### Put your parsed results into a DataFrame

In [107]:
yelp_resturants = pd.DataFrame(yelp_resturants)

yelp_resturants.head()

,resturant_name,categories,address,review_count,rating,station_name,distance
0,La Taverna di San Giuseppe,Tuscan,Via Giovanni Dupre 132,169,4.8,Curtatone,745.395007
1,Zest,"Italian, Wine Bars",Costa di Sant'antonio 13,26,4.7,Curtatone,177.848119
2,Compagnia dei Vinattieri,Tuscan,"Via delle Terme, 79",24,4.3,Curtatone,231.846606
3,La Chiacchiera,Italian,Via Costa di San Antonio 4,23,4.3,Curtatone,181.200690
4,Café Nannini,Cafes,Piazza Matteotti 32,14,4.4,Curtatone,60.451222


# Comparing Results

#### Which API provided you with more complete data? Provide an explanation. 

In [108]:
print(f'Yelp Shape: {yelp_resturants.shape}')
print(f'Foursquare Shape: {foursquare_resturants.shape}')

Yelp Shape: (220, 7)
Foursquare Shape: (168, 5)


The Yelp API includes more detailed data as well as has higher number of resturants compared to the Foursquare API. With this information the Yelp API provides better coverage of data than the Foursquare API.

#### Get the top 10 restaurants according to their rating

In [109]:
yelp_resturants.sort_values(['rating', 'review_count'], ascending=False).head(10)

,resturant_name,categories,address,review_count,rating,station_name,distance
176,Mugolone,"Italian, Gluten-Free",Via dei Pellegrini 8,7,5.0,Casa dell'Ambiente,968.765622
206,Mugolone,"Italian, Gluten-Free",Via dei Pellegrini 8,7,5.0,Mercato,253.346444
102,Albergo Certosa di Maggiano,"Hotels & Travel, Restaurants","Str. di Certosa, 82",2,5.0,Due Ponti,822.545866
149,La Mandorla,"Desserts, Patisserie/Cake Shop, Ice Cream & Fr...",Via Camollia 36,2,5.0,Terminal Bus,906.468202
186,BAR Il Barrino,Bars,"Str. Massetana Romana, 5",2,5.0,Frajese,391.175672
99,Buti pasticceria,"Chocolatiers & Shops, Desserts","Viale Vittorio Emanuele II, 53",1,5.0,Antiporto,178.490110
101,Havana Club,"Cuban, Steakhouses",Piazza Maestri del Lavoro 27,1,5.0,Due Ponti,217.539983
106,Antica Trattoria Botteganova,"Italian, Bars",Strada Chiantigiana 29,1,5.0,Napoli,659.400589
145,La Grotta di San Francesco,"Seafood, Tuscan",Via Camollia 25,1,5.0,Terminal Bus,1000.171874
150,Pasta Mondo,"Pizza, Italian, Pasta Shops",Via Camollia 173,1,5.0,Terminal Bus,660.627489


#### Turn API DataFrame's into CSV's

In [110]:
foursquare_resturants.to_csv('foursquare_resturants.csv', index=False)

yelp_resturants.to_csv('yelp_resturants.csv', index=False)